In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
def print_grid(data):
    cells = np.zeros((8,8))
    for item in data:
        cells[item[0]-1][item[1]-1] = 1
    for row in cells:
        print(" ".join(["X" if item == 1 else "O" for item in row]))
def check_mutual_cells(data,column_number):
    counter = 0
    que     = []
    for item in data:
        if item[column_number] in que:
            counter += 1
        else:
            que.append(item[column_number])
    return counter
def count_row_threats(data):
    return check_mutual_cells(data,0)
def count_column_threats(data):
    return check_mutual_cells(data,1)
def count_diagonal_threats(data):
    counter = 0
    cells = np.zeros((8,8))
    for item in data:
        x = item[0]-1
        y = item[1]-1
        if cells[x][y] == 1:
            counter += 1
        for i in range(-8,8):
            if 0<=x+i<=7:
                if 0<=y+i<=7:
                    cells[x+i][y+i] = 1
                if 0<=y-i<=7:
                    cells[x+i][y-i] = 1
    return counter
def count_all(data):
    return count_row_threats(data) + count_column_threats(data) + count_diagonal_threats(data)
def check_condtions(data):
    if count_row_threats(data)==0:
        if count_column_threats(data)==0:
            if count_diagonal_threats(data)==0:
                return True
    return False
def check_exist(data,x,y):
    for i in range(8):
        if data[i][0] == x and data[i][1] == y:
            return False
    return True
def sub_move(data,index,direction,diff_x,diff_y,mulitply):
    if direction == 0 or direction == 8:
        diff_x = -mulitply
    elif direction == 2:
        diff_y = mulitply
    elif direction == 4:
        diff_x = mulitply
    else:
        diff_y = -mulitply
    return diff_x,diff_y
def base_move(data,index,direction,mulitply):
    diff_x = 0
    diff_y = 0
    if direction%2 == 0:
        diff_x,diff_y = sub_move(data,index,direction,diff_x,diff_y,mulitply)
    else:
        diff_x,diff_y = sub_move(data,index,direction-1,diff_x,diff_y,mulitply)
        diff_x,diff_y = sub_move(data,index,direction+1,diff_x,diff_y,mulitply)
    new_x = data[index][0] + diff_x
    new_y = data[index][1] + diff_y
    return new_x,new_y
def pure_move(data,index,direction,mulitply=1):
    new_x,new_y = base_move(data,index,direction,mulitply)
    data[index][0] = new_x
    data[index][1] = new_y
def move(data,index,direction,mulitply=1):
    new_x,new_y = base_move(data,index,direction,mulitply)
    if 0<new_x<9 and 0<new_y<9:
        if check_exist(data,new_x,new_y):
            data[index][0] = new_x
            data[index][1] = new_y
            return True
    return False
def base_super_move(data,index,direction,mulitply):
    length = 2*abs(mulitply)
    other = direction%length - abs(mulitply)
    if mulitply<0:
        other = -other
    if direction < length:
        diff_x = other
        diff_y = mulitply
    elif direction < 2*length:
        diff_x = mulitply
        diff_y = -other
    elif direction < 3*length:
        diff_x = -other
        diff_y = -mulitply
    else:
        diff_x = -mulitply
        diff_y = other
    new_x = data[index][0] + diff_x
    new_y = data[index][1] + diff_y
    return new_x,new_y
def pure_super_move(data,index,direction,mulitply=1):
    new_x,new_y = base_super_move(data,index,direction,mulitply)
    data[index][0] = new_x
    data[index][1] = new_y
def super_move(data,index,direction,mulitply=1):
    new_x,new_y = base_super_move(data,index,direction,mulitply)
    if 0<new_x<9 and 0<new_y<9:
        if check_exist(data,new_x,new_y):
            data[index][0] = new_x
            data[index][1] = new_y
            return True
    return False

تابع کمکی سرچ

In [2]:
def sub_increament(numbers,maxi,i):
    numbers[-i] += 1
    if numbers[-i] == maxi-i:
        sub_increament(numbers,maxi,i+1)
        numbers[-i] = numbers[-(i+1)] + 1
def increament(numbers,maxi):
    try:
        sub_increament(numbers,maxi+1,1)
        return True
    except:
        return False
def create_list(begin,end):
    return [i for i in range(begin,end)]
def convert_to_move(numbers,length):
    arr = [0]*length
    for n in numbers:
        arr[n] = 1
    temp = 0
    ret  = []
    for i in arr:
        if i == 0:
            ret.append(temp)
            temp = 0
        else:
            temp+=1
    ret.append(temp)
    return ret

پیاده سازی الگوریتم های سرچ 

In [3]:
def idfs(data,level):
    if level == 0:
        if check_condtions(data):
            return data
        else:
            return None
    for index in range(8):
        for direction in range(8):
            if move(data,index,direction):
                ret = idfs(data,level-1)
                if ret is not None:
                    return ret
                pure_move(data,index,direction,-1)
    return None
def sub_myidfs(data,moves,index):
    next_index = 0
    for next_index in range(index+1,9):
        if next_index == 8:
            break
        if moves[next_index] != 0:
            break
    for direction in range(8*moves[index]):
        if super_move(data,index,direction,moves[index]):
            if next_index == 8:
                if check_condtions(data):
                    return data
            else:
                ret = sub_myidfs(data,moves,next_index)
                if ret is not None:
                    return ret
            pure_super_move(data,index,direction,-moves[index])
    return None
def my_idfs(data,level):
    if level == 0:
        if check_condtions(data):
            return data
        else:
            return None
    n = create_list(0,level)
    while True:
        moves = convert_to_move(n,7+level)
        ret = sub_myidfs(data,moves,n[0])
        if ret is not None:
            return ret
        if not increament(n,7+level):
            break
    return None

In [4]:
for fn in range(1,5):
    data = pd.read_csv('in'+str(fn)+'.csv',header=None)
    data = data.values.tolist()
    begin = time.time()
    for i in range(0,8):
        ret = my_idfs(data,i)
        print(ret)
        if ret is not None:
            print_grid(data)
            break
    end = time.time()
    print(end-begin)

None
None
None
None
None
None
[[1, 4], [2, 2], [3, 5], [4, 8], [5, 6], [6, 1], [7, 3], [8, 7]]
O O O X O O O O
O X O O O O O O
O O O O X O O O
O O O O O O O X
O O O O O X O O
X O O O O O O O
O O X O O O O O
O O O O O O X O
18.67993688583374
None
None
None
None
None
None
[[1, 5], [3, 2], [2, 7], [4, 6], [5, 3], [6, 1], [8, 4], [7, 8]]
O O O O X O O O
O O O O O O X O
O X O O O O O O
O O O O O X O O
O O X O O O O O
X O O O O O O O
O O O O O O O X
O O O X O O O O
37.74926996231079
None
None
None
None
None
None
None
[[1, 4], [4, 3], [2, 6], [3, 8], [5, 1], [6, 7], [8, 2], [7, 5]]
O O O X O O O O
O O O O O X O O
O O O O O O O X
O O X O O O O O
X O O O O O O O
O O O O O O X O
O O O O X O O O
O X O O O O O O
109.4988648891449
None
None
None
None
[[1, 4], [2, 7], [3, 3], [4, 8], [5, 2], [6, 5], [7, 1], [8, 6]]
O O O X O O O O
O O O O O O X O
O O X O O O O O
O O O O O O O X
O X O O O O O O
O O O O X O O O
X O O O O O O O
O O O O O X O O
0.2911107540130615
